<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Quantum Process Tomography

* **Last Updated:** April 29, 2019
* **Requires:** qiskit-terra 0.8, qiskit-ignis 0.1.1, qiskit-aer 0.2

This notebook contains examples for using the ``ignis.verification.tomography`` process tomography module.

In [1]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.tools.qi.qi import outer

# Tomography functions
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter

## 1-qubit process tomography example

In [2]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=4000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp',), ('X',)): {'0': 4000},
 (('Zp',), ('Y',)): {'0': 1980, '1': 2020},
 (('Zp',), ('Z',)): {'0': 2053, '1': 1947},
 (('Zm',), ('X',)): {'1': 4000},
 (('Zm',), ('Y',)): {'0': 2064, '1': 1936},
 (('Zm',), ('Z',)): {'0': 2018, '1': 1982},
 (('Xp',), ('X',)): {'0': 1982, '1': 2018},
 (('Xp',), ('Y',)): {'0': 2023, '1': 1977},
 (('Xp',), ('Z',)): {'0': 4000},
 (('Yp',), ('X',)): {'0': 1944, '1': 2056},
 (('Yp',), ('Y',)): {'1': 4000},
 (('Yp',), ('Z',)): {'0': 1986, '1': 2014}}

In [3]:
# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_lstsq = qpt_tomo.fit(method='lstsq')
print('Least-Sq Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_lstsq.data / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = qpt_tomo.fit(method='cvx')
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx.data / 2))

Least-Sq Fitter
fit time: 0.025106191635131836
fit fidelity: 0.9935435801308409

CVXOPT Fitter
fit time: 0.14531993865966797
fit fidelity: 0.9999047428166897


## 1-qubit process tomography of two-qubit swap gate

We will prepare qubit-0 and measure qubit-1 so the reconstructed channel should be an identity

In [4]:
# Process tomography of a Hadamard gate
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.swap(q[0], q[1])

# Ideal channel is a unitary
ideal_unitary = np.eye(2)
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
# We use the optional prepared_qubits kwarg to specify that the prepared qubit was different to measured qubit
qpt_circs = process_tomography_circuits(circ, q[1], prepared_qubits=q[0])
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp',), ('X',)): {'0': 985, '1': 1015},
 (('Zp',), ('Y',)): {'0': 1029, '1': 971},
 (('Zp',), ('Z',)): {'0': 2000},
 (('Zm',), ('X',)): {'0': 981, '1': 1019},
 (('Zm',), ('Y',)): {'0': 1016, '1': 984},
 (('Zm',), ('Z',)): {'1': 2000},
 (('Xp',), ('X',)): {'0': 2000},
 (('Xp',), ('Y',)): {'0': 955, '1': 1045},
 (('Xp',), ('Z',)): {'0': 958, '1': 1042},
 (('Yp',), ('X',)): {'0': 989, '1': 1011},
 (('Yp',), ('Y',)): {'0': 2000},
 (('Yp',), ('Z',)): {'0': 1003, '1': 997}}

In [5]:
# Least-Squares tomographic reconstruction
t = time.time()
choi_lstsq = qpt_tomo.fit(method='lstsq')
print('Least-Sq Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_lstsq.data / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = qpt_tomo.fit(method='cvx')
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx.data / 2))

Least-Sq Fitter
fit time: 0.009311914443969727
fit fidelity: 0.9874012369433787

CVXOPT Fitter
fit time: 0.06017184257507324
fit fidelity: 0.9998944136683781


## 2-Qubit entangling circuit

In [6]:
# Bell-state entangling circuit
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.h(q[0])
circ.cx(q[0], q[1])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)

t = time.time()
choi_lstsq = qpt_tomo.fit(method='lstsq')
print('Least-Sq Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_lstsq.data / 4))

t = time.time()
choi_cvx = qpt_tomo.fit(method='cvx')
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_cvx.data / 4))

Least-Sq Fitter
fit time: 0.14422106742858887
fit fidelity: 0.984365187084024

CVXOPT Fitter
fit time: 1.1474080085754395
fit fidelity: 0.9998986962593072


## Using SIC-POVM preparation basis

In [7]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q, prep_labels='SIC', prep_basis='SIC')
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs, prep_basis='SIC')
qpt_tomo.data

{(('S0',), ('X',)): {'0': 2000},
 (('S0',), ('Y',)): {'0': 995, '1': 1005},
 (('S0',), ('Z',)): {'0': 1027, '1': 973},
 (('S1',), ('X',)): {'0': 682, '1': 1318},
 (('S1',), ('Y',)): {'0': 1049, '1': 951},
 (('S1',), ('Z',)): {'0': 1952, '1': 48},
 (('S2',), ('X',)): {'0': 674, '1': 1326},
 (('S2',), ('Y',)): {'0': 1832, '1': 168},
 (('S2',), ('Z',)): {'0': 504, '1': 1496},
 (('S3',), ('X',)): {'0': 683, '1': 1317},
 (('S3',), ('Y',)): {'0': 182, '1': 1818},
 (('S3',), ('Z',)): {'0': 512, '1': 1488}}

In [8]:
# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_lstsq = qpt_tomo.fit(method='lstsq')
print('Least-Sq Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_lstsq.data / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = qpt_tomo.fit(method='cvx')

print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx.data / 2))

Least-Sq Fitter
fit time: 0.008871316909790039
fit fidelity: 0.9969739538161556

CVXOPT Fitter
fit time: 0.0575709342956543
fit fidelity: 0.9995559180011946
